### Data wrangling for LAB SESSION 3


### 1. Importing all the libraries:


In [1]:
import pandas as pd
import json
import requests
import urllib
import pandas_datareader as pdr

### 2. Reading the data file in a dataframe (df):

In [106]:
df = pd.read_excel('lab3.xlsx')

### 3. Some changes in column names:

In [107]:
df = df.rename(columns={
                        'YEAR':'Year',
                        'records lost':'Records Lost',
                        'ORGANISATION':'Organisation',
                        'METHOD OF LEAK':'Method Of Leak',
                        'DATA SENSITIVITY':'Data Sensitivity',
                        })

### 4. To access Yahoo finance data for the ticker symbols of all the companies:

In [110]:
import urllib
import json
import requests

In [112]:

def get_symbol(name):
    url = 'http://d.yimg.com/autoc.finance.yahoo.com/autoc?query='+name+'&region=1&lang=en'

    result = requests.get(url).json()
    
    for x in result['ResultSet']['Result']:
        return x['symbol']

ticker=[]
company = df.Entity
 
for tkr in company:
    ticker.append(get_symbol(tkr))
    
print(ticker)

[u'AOL-U.TI', u'ADP', None, u'CIT-U.TI', None, u'HPQ', None, u'KDDIY', None, u'AOL-U.TI', None, None, None, u'GPS', None, None, None, None, None, u'AMTD', u'FIS', u'DNPCF', None, None, None, None, None, u'AT', u'SBUX', None, None, None, u'^UMIAMIFL', None, None, None, None, None, None, None, None, None, None, None, None, u'LYNS', None, None, u'IGRW', None, None, None, None, u'HTLD', None, u'AT', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, u'PPB.L', u'JPM-PB', None, None, None, None, None, None, None, None, None, None, None, None, None, None, u'CIT-U.TI', None, None, u'GB00B1Z54R10.L', None, None, None, u'SGAMY', None, None, None, None, None, None, None, u'NHS', None, None, None, None, u'SAND-P.V', None, None, None, None, None, None, None, None, None, u'GPN', None, None, None, None, None, u'KTC.DU', None, None, u'ATVI', None, None, None, None, u'KCT.V', u'K55306BA2822.KS', None, None, None, None, None, None, u'CIT-U.TI', None, None, u'2GF.F'

### 5. Saving all the ticker symbols in a new dataframe - ticker_df

In [113]:
ticker_df = pd.DataFrame(ticker)

In [114]:
ticker_df.head()

,0
0,AOL-U.TI
1,ADP
2,None
3,CIT-U.TI
4,None


--> To link the stock price data to time:

In [115]:
import pandas_datareader as pdr

In [117]:
import datetime
start_date = datetime.datetime(2004,1,1)
end_date = datetime.datetime(2017,6,6)
source = 'yahoo'
panel_data = pdr.DataReader(ticker, source, start_date, end_date)

In [118]:
panel_data

<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 3567 (major_axis) x 63 (minor_axis)
Items axis: Adj Close to Volume
Major_axis axis: 2004-01-01 00:00:00 to 2017-06-07 00:00:00
Minor_axis axis: None to ^UMIAMIFL

--> Putting the panel data into the data frame:

In [121]:
close = panel_data.ix['Close']

--> to include only weekdays:

In [122]:
weekdays = pd.date_range(start=start, end=end, freq='B')

In [123]:
close = close.reindex(weekdays)

In [126]:
close = close.dropna(axis=1,how='all')

In [127]:
ticker_df = ticker_df.rename(columns={
                        0:'Ticker Symbol'})

In [128]:
ticker_df.head()

,Ticker Symbol
0,AOL-U.TI
1,ADP
2,None
3,CIT-U.TI
4,None


Adding the ticker symbol to the main data frame - df :

In [129]:
df['Ticker Symbol'] = ticker_df['Ticker Symbol']

In [130]:
df.head()

,Entity,Year,Records Lost,Organisation,Method Of Leak,Data Sensitivity,Ticker Symbol
0,AOL,2004,92000000,web,inside job,Just email address/Online information,AOL-U.TI
1,Automatic Data Processing,2005,125000,financial,poor security,SSN/Personal details,ADP
2,Ameritrade Inc.,2005,200000,financial,lost / stolen device,SSN/Personal details,None
3,Citigroup,2005,3900000,financial,lost / stolen device,Credit card information,CIT-U.TI
4,Cardsystems Solutions Inc.,2005,40000000,financial,hacked,Credit card information,None


In [131]:
df = df.dropna(axis=0,how='any')

Was trying to reformat my table using this (mentioned below) code which took forever to run so tried something else - 

As I first transposed my table and then I had remove the transpose to actually get this to work.



In [134]:
##r_df= pd.DataFrame(columns=['Col1','Col2','col3'])
    #k =0
#for i in range(len(close)) : 
    #ts = close.index[i]
    #for j in range(len(close.iloc[i])): 
        #time_frame = pd.to_datetime(str(close.columns.values[j]))
        #date = time_frame.strftime('%Y-%m-%d')
        #r_df.loc[k]=( ts,  date , close.iloc[i][j])
        #k=k+1

In [136]:
close.head()

,0303.HK,035080.KQ,2GF.F,AABA,AAPL,ADBE,ADP,AMTD,ANTM,AT,...,T,TALK.L,TGT,TWTR,UBI.PA,UPS,VKTX,VOD,YAHOY,^CEX
2004-01-01,11.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.5900,NaN,NaN,NaN,NaN,NaN
2004-01-02,11.00,2543.219971,NaN,22.700001,1.520000,19.549999,31.690842,14.15,38.139999,NaN,...,26.139999,NaN,37.720001,NaN,5.6125,73.650002,NaN,29.485947,NaN,197.270004
2004-01-05,11.60,2713.330078,NaN,23.450001,1.583571,19.900000,32.626656,14.88,36.855000,NaN,...,27.049999,NaN,37.400002,NaN,5.8125,73.930000,NaN,30.650940,NaN,200.089996
2004-01-06,11.55,2721.840088,NaN,23.620001,1.577857,19.920000,32.935951,15.00,37.055000,NaN,...,27.120001,NaN,38.570000,NaN,5.6775,73.480003,NaN,31.116936,NaN,198.300003
2004-01-07,11.70,2768.620117,NaN,23.834999,1.613571,19.719999,32.761478,15.14,37.250000,NaN,...,27.100000,NaN,38.270000,NaN,5.6975,73.239998,NaN,31.000437,NaN,197.759995


--> to decrease the data size, I came up with an average stock price for all the companies for one year:

In [137]:
close = close.groupby(pd.TimeGrouper(freq='12M')).mean()

--> Transposing the original table:

In [138]:
close = close.T

In [139]:
close.head()

,2004-01-31 00:00:00,2005-01-31 00:00:00,2006-01-31 00:00:00,2007-01-31 00:00:00,2008-01-31 00:00:00,2009-01-31 00:00:00,2010-01-31 00:00:00,2011-01-31 00:00:00,2012-01-31 00:00:00,2013-01-31 00:00:00,2014-01-31 00:00:00,2015-01-31 00:00:00,2016-01-31 00:00:00,2017-01-31 00:00:00,2018-01-31 00:00:00
0303.HK,11.956818,13.152299,19.583469,38.933400,59.039634,38.519918,54.180478,83.305400,84.078189,90.429000,105.294262,100.252632,96.951220,89.317073,98.114881
035080.KQ,2758.696682,2051.893008,4614.145226,7148.666972,5795.079755,4404.571429,6234.212598,5103.869048,5319.190283,6211.400000,9145.510204,11107.398374,10496.048387,6609.085366,5116.726190
2GF.F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.730577,0.414558,0.123178,0.029996,0.007642,0.009451,0.015126
AABA,23.770250,30.758254,35.605357,29.151554,27.017421,20.367857,15.215179,15.589486,15.532698,16.342271,29.362619,40.140359,37.581315,38.563399,47.285632
AAPL,1.628678,2.800831,7.157752,10.241833,19.153662,19.460459,22.292686,38.566522,53.023271,83.036505,67.990975,95.024644,119.152988,106.250830,143.168390


--> to reformat the table to a better format which is more usable and efficient:

In [143]:
final_df= pd.DataFrame(columns=['Ticker symbol','Date','Price'])
k =0
for i in range(len(close)) : 
    ts = close.index[i]
    for j in range(len(close.iloc[i])): 
        time_frame = pd.to_datetime(str(close.columns.values[j]))
        date = time_frame.strftime('%Y-%m-%d')
        final_df.loc[k]=( ts,  date , close.iloc[i][j])
        k=k+1

--> converting the file to CSV:

In [146]:
final_df.to_csv('Lab3_Data_Final.csv', index=False, encoding='utf-8')